### Robust Gaussian Covariance Matrix Estimation

Given $m$ observations $a_1,..., a_m$ of a zero-mean random $n$ - vector $A = (A_1, ..., A_n)$, we would like to find the maximum likelihood inverse covariance matrix $\Sigma^{-1}$. Taking the log-likelihood of the data, we get:

\begin{equation*}
  \begin{aligned}
    &\text{minimize} && - \log \det \Sigma^{-1} + \sum_{i = 1}^m \text{tr}(\Sigma^{-1} a_i a_i^T) \\
  \end{aligned}
\end{equation*}

We make a slight modification to this problem: instead of taking all of the data, we would like to minimize the sum of the $k$ largest values of 

\begin{equation}
-\log \det \Sigma^{-1} + \sum_{i = 1}^m \text{tr}(\Sigma^{-1} a_i a_i^T)
\end{equation}

This corresponds to selecting the $k$ "worst-case" observations $a_i$ and minimizing those. Let $R = \Sigma^{-1}$. Letting $z$ be the vector such that:

\begin{equation}
z_i  = -\log \det R + \text{tr}(R a_i a_i^T)
\end{equation}

We get the problem:

\begin{equation*}
  \begin{aligned}
    &\text{minimize} && \sum_{j = 1}^3 z_{[j]} \\
  \end{aligned}
\end{equation*}

where $z_{[1]}$ is the largest element of a vector, $z_{[2]}$ is the second largest, and so on. This is a convex optimization problem with variable $R$ and constants $a_i$.

It is equivalently formulated in epigraph form as:

\begin{equation*}
  \begin{aligned}
    &\text{minimize} && t + t_{det}\mathbb{1} \\
    &\text{subject to} &&  -\log \det R \leq t_{det} \\
                   &   &&  t_i = \text{tr}\, (R a_i a_i^T) &&& i = 1,..., m \\
  \end{aligned}
\end{equation*}

where $t \in \mathbb{R}^n$, and $t_{det} \in \mathbb{R}$ are the variables and $a_i$ are the constants.




In [30]:
import cvxpy as cp
import numpy as np
import scipy as sp

# Variable declarations

np.random.seed(0)
m = 11 # Number of observations of each random variable
n = 5 # Number of random variables
k = 3 # Needs to be less than m. 
A = np.matrix(np.random.rand(m,n))
A -= np.mean(A, axis=0)
K = np.array([(A[i].T*A[i]).flatten() for i in range(m)])


# Problem construction
problems = []
opt_vals = []

# Problem 1 (Epigraph formulation)
sigma_inv1 = cp.Variable(n,n) # Inverse covariance matrix
t = cp.Variable(m)
tdet = cp.Variable(1)

f = cp.sum_largest(t+tdet, k)
z = K*cp.reshape(sigma_inv1, n*n, 1)
C = [-cp.log_det(sigma_inv1) <= tdet, t == z]
problems.append(cp.Problem(cp.Minimize(f), C))
opt_vals.append(None)

# Problem 2 (Equivalent unconstrained formulation)
sigma_inv2 = cp.Variable(n, n) # Inverse covariance matrix
obs = cp.vstack([-cp.log_det(sigma_inv2) + cp.trace(A[i].T*A[i]*sigma_inv2) for i in range(m)])
f2 = cp.sum_largest(obs, k)
problems.append(cp.Problem(cp.Minimize(f2)))
opt_vals.append(None)

# For debugging individual problems:
if __name__ == "__main__":
    for prob, opt_val in zip(problems, opt_vals):
        prob.solve(solver = "SCS", eps=1e-5)
        print("status:", prob.status)
        print("optimal value:", prob.value)
        print("true optimal value:", opt_val)
        for var in prob.variables():
            print(var.value)


('status:', 'optimal_inaccurate')
('optimal value:', -24.88493554964564)
('true optimal value:', None)
[[ 2.55453163]
 [ 2.37275358]
 [ 3.19224176]
 [ 4.99979084]
 [ 3.73921866]
 [ 4.89245726]
 [ 4.99978692]
 [ 4.99980468]
 [ 4.99979773]
 [ 4.99979373]
 [ 4.99979108]]
-13.2947817453
[[ 16.94992841  -0.44395379   2.08884349  -0.74265984   4.98032542]
 [ -0.44395387  29.54417678  -4.60108393  -3.21451982  19.05715578]
 [  2.0888435   -4.60108395  14.83020547  -0.21003973  -6.00186905]
 [ -0.74265985  -3.21451984  -0.21003974   9.06997558  -3.32435687]
 [  4.98032533  19.05715578  -6.00186904  -3.32435688  24.93792394]]
('status:', 'optimal_inaccurate')
('optimal value:', -24.80359504396053)
('true optimal value:', None)
[[ 17.6041589    0.04570286   1.90762769  -0.39874142   5.44784174]
 [  0.04570248  25.38041942  -3.43957074  -2.36351938  16.87592648]
 [  1.90762781  -3.43957061  14.23912861  -0.18830089  -5.37854083]
 [ -0.39874187  -2.36351993  -0.18830061   9.44828135  -2.70745568]


In [46]:
import numpy as np
import cvxpy as cp
import scipy as sp

import scipy.sparse as sps

n = 10000
m = 2000

s = int(np.ceil(n//10))
x_true = np.vstack((np.random.randn(s, 1),np.zeros((n-s, 1))))
x_true = np.random.permutation(x_true)

density = 0.1
rcA = 0.1
A = sps.random(m, n, density, data_rvs = np.random.randn)

In [51]:
np.linalg.cond(A.todense())

2.629704017598681